## Análise dos dados dos covenants coletados

In [1]:
# Bibliotecas utilizadas
import os
import pandas as pd
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
import fitz  # PyMuPDF para leitura de PDFs


ModuleNotFoundError: No module named 'spacy'

In [ ]:
# **1. Carregar e pré-processar dados manuais**
df_covenants = pd.read_csv("base_covenants.csv", delimiter=";", encoding="utf-8")
df_covenants.columns = df_covenants.columns.str.strip()

# **2. Carregar NLP do spaCy**
nlp = spacy.load("pt_core_news_sm")

def preprocessar_texto(texto):
    """ Limpa e tokeniza usando spaCy """
    doc = nlp(texto.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return " ".join(tokens)

df_covenants["Texto Processado"] = df_covenants["Descrição"].apply(preprocessar_texto)

# **3. Vetorização TF-IDF**
vectorizer = TfidfVectorizer(max_features=500)
X = vectorizer.fit_transform(df_covenants["Texto Processado"])

# **4. Codificar labels**
encoder = LabelEncoder()
y = encoder.fit_transform(df_covenants["TIPO"])

# **5. Treinar modelo de classificação**
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# **6. Função para extrair informações específicas**
def extrair_texto_pdf(caminho_pdf):
    """ Extrai texto de um PDF """
    texto = ""
    with fitz.open(caminho_pdf) as pdf:
        for pagina in pdf:
            texto += pagina.get_text("text") + "\n"
    return texto

def extrair_limite(texto):
    """ Extrai valores numéricos relacionados a limites do texto """
    match = re.search(r"limite.*?(\d+[.,]?\d*)", texto, re.IGNORECASE)
    return match.group(1) if match else "Não especificado"

def verificar_violacao(texto):
    """ Detecta termos indicativos de violação """
    padroes_violacao = ["violação", "descumprimento", "inadimplência", "não cumprimento"]
    return "Sim" if any(p in texto.lower() for p in padroes_violacao) else "Não"

def verificar_divulgacao(texto):
    """ Verifica se o texto menciona divulgação do covenant """
    return "Sim" if "divulgação" in texto.lower() else "Não"

In [ ]:
# **7. Processar PDFs novos**
covenants_notas = "M:\\Maria Eduarda\\Downloads\\Empresas Com Covenants"
dados = []

for caminho, subpastas, arquivos in os.walk(covenants_notas):
    if caminho == covenants_notas:
        continue
    
    relative_path = os.path.relpath(caminho, covenants_notas)
    empresa = relative_path.split(os.sep)[0]
    
    for arquivo in arquivos:
        if arquivo.endswith(".pdf"):
            ano = re.search(r"\d{4}", arquivo)
            ano = int(ano.group()) if ano else "Desconhecido"
            
            # **8. Ler e processar texto**
            texto = extrair_texto_pdf(os.path.join(caminho, arquivo))
            texto_processado = preprocessar_texto(texto)
            X_novo = vectorizer.transform([texto_processado])
            
            # **9. Predizer tipo de covenant**
            tipo_predito = encoder.inverse_transform(modelo.predict(X_novo))[0]
            
            # **10. Extração de informações adicionais**
            limite = extrair_limite(texto)
            violou = verificar_violacao(texto)
            divulgou = verificar_divulgacao(texto)
            possui_covenant = "Sim" if tipo_predito else "Não"
            
            # **11. Salvar no DataFrame**
            dados.append({
                "EMPRESA": empresa,
                "POSSUI COVENANT": possui_covenant,
                "DIVULGOU": divulgou,
                "ANO": ano,
                "Debenture ou emprestimo e financiamento": "Debenture" if "debenture" in texto.lower() else "Financiamento",
                "TIPO": tipo_predito,
                "Limite": limite,
                "Violou?": violou
            })

In [ ]:
# **12. Criar e salvar DataFrame final**
df_resultado = pd.DataFrame(dados)
df_resultado.to_excel("covenants_detectados.xlsx", index=False)

# Exibir amostra
display(df_resultado)